In [36]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, Table, Column, Integer, String, LargeBinary, Text, MetaData, DateTime
import json


with open('db-config.json') as f:
    config = json.load(f)

user = config['user']
password = config['password']
host = config['host']
port = config['port']
database = config['database']

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}",
                       connect_args={"connect_timeout": 30, "read_timeout": 120, "write_timeout": 120},
                        pool_pre_ping=True)

In [37]:
# air_quality_dataset 테이블 데이터 읽어오기
query = "SELECT * FROM air_quality_dataset"
df = pd.read_sql(query, con=engine)

In [38]:
from sqlalchemy import Table, Column, Integer, String, LargeBinary, DateTime, MetaData

metadata = MetaData()

models_table = Table('models', metadata, autoload_with=engine)

In [39]:
from sqlalchemy import select
from keras.models import load_model
from io import BytesIO
import tempfile
import os

def load_latest_model(conn, target, region):
    query = models_table.select().where(
        models_table.c.name == f"lstm_{target}_{region}"
    ).order_by(models_table.c.created_at.desc()).limit(1)

    result = conn.execute(query).mappings().fetchone()
    if result is None:
        raise ValueError(f"No model found for {target} - {region}")

    model_binary = result['data']

    # 🔧 임시 파일에 저장
    with tempfile.NamedTemporaryFile(suffix=".h5", delete=False) as tmp:
        tmp.write(model_binary)
        tmp.flush()
        tmp_path = tmp.name  # 경로 저장

    try:
        model = load_model(tmp_path, compile=False)
        model.compile(optimizer='adam', loss='mse')
    finally:
        os.remove(tmp_path)  # 사용 후 삭제

    return model

In [40]:
def get_input_data_for_region(region, window_size=30):
    df_region = df[df['region'] == region].sort_values('datetime', ascending=False).head(window_size)

    if len(df_region) < window_size:
        raise ValueError(f"{region} 데이터 부족: {len(df_region)} rows")

    X = df_region[feature_cols].iloc[::-1].values  # 시간순 정렬 (과거 → 현재)
    return X.reshape(1, window_size, len(feature_cols))

In [41]:
def generate_prediction(model, input_data, output_len=31):
    pred = model.predict(input_data)
    pred = pred.reshape(-1)[:output_len]
    return pred

In [42]:
def save_predictions(conn, region, start_datetime, pm10_preds, pm25_preds):
    data = []
    for i in range(len(pm10_preds)):
        dt = start_datetime + timedelta(days=i)
        row = {
            'datetime': dt,
            'region': region,
            'pm10': float(pm10_preds[i]),
            'pm25': float(pm25_preds[i])
        }
        data.append(row)

    # ✅ 여러 행을 한 번에 insert (list of dicts)
    conn.execute(pred_table.insert(), data)
    conn.commit()

In [43]:
with engine.connect() as conn:
    region_list = pd.read_sql("SELECT DISTINCT region FROM air_quality", conn)['region'].tolist()

In [44]:
feature_cols = ['aod_avg', 'wind_speed', 'precipitation']
target_cols = ['pm10', 'pm25']

In [45]:
from sqlalchemy import MetaData, Table

metadata = MetaData()

pred_table = Table(
    "air_quality_day_pred", metadata, autoload_with=engine
)

In [46]:
from datetime import datetime, timezone, timedelta
output_len = 31
KST = timezone(timedelta(hours=9))
start_date = datetime.now(KST).replace(hour=0, minute=0, second=0, microsecond=0)

with engine.connect() as conn:
    for region in region_list:  # 예: ['강남구', '서초구', ...]
        try:
            print(f"\n🚀 {region} 예측 시작")

            # 1. 모델 로드 (DB에서 최신 모델)
            pm10_model = load_latest_model(conn, "pm10", region)
            pm25_model = load_latest_model(conn, "pm25", region)

            # 2. 입력 데이터
            input_data = get_input_data_for_region(region)

            # 3. 예측
            pm10_preds = generate_prediction(pm10_model, input_data, output_len)
            pm25_preds = generate_prediction(pm25_model, input_data, output_len)

            # 4. 저장
            save_predictions(conn, region, start_date + timedelta(days=1), pm10_preds, pm25_preds)

            print(f"✅ {region} 예측 완료 및 저장")

        except Exception as e:
            print(f"❌ {region} 실패: {e}")



🚀 강남구 예측 시작


E0000 00:00:1751474987.473135 23469208 meta_optimizer.cc:967] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 517ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step
✅ 강남구 예측 완료 및 저장

🚀 강동구 예측 시작
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
✅ 강동구 예측 완료 및 저장

🚀 강북구 예측 시작
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
✅ 강북구 예측 완료 및 저장

🚀 강서구 예측 시작


E0000 00:00:1751475059.065720 23469208 meta_optimizer.cc:967] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
✅ 강서구 예측 완료 및 저장

🚀 관악구 예측 시작
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
✅ 관악구 예측 완료 및 저장

🚀 광진구 예측 시작
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step
✅ 광진구 예측 완료 및 저장

🚀 구로구 예측 시작


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
✅ 양천구 예측 완료 및 저장

🚀 영등포구 예측 시작
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step
✅ 영등포구 예측 완료 및 저장

🚀 용산구 예측 시작
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step
✅ 용산구 예측 완료 및 저장

🚀 은평구 예측 시작


E0000 00:00:1751475479.075698 23469208 meta_optimizer.cc:967] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
✅ 은평구 예측 완료 및 저장

🚀 종로구 예측 시작
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
✅ 종로구 예측 완료 및 저장

🚀 중구 예측 시작
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
✅ 중구 예측 완료 및 저장

🚀 중랑구 예측 시작


E0000 00:00:1751475549.256585 23469208 meta_optimizer.cc:967] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
✅ 중랑구 예측 완료 및 저장

🚀 평균 예측 시작
❌ 평균 실패: No model found for pm10 - 평균
